## Eviroment Setup
bin2coe is needed to generate the coe file for simulating the program on the desktop

compilerMagic is needed to compile the C program cell

In [ ]:
!pip3 install bin2coe

In [1]:
%load_ext compilerMagic

## Program Cell
The cell below contains the C code of the program

In [2]:
%%riscvc blink_upper_LEDs

#include <stdint.h>

const int value = 0;
const int tristate = 1;
int32_t volatile * const buttons = (int32_t*) 0x40010000;
int32_t volatile * const LEDs    = (int32_t*) 0x40020000;

int main()
{
  // Turn off all LEDs
  LEDs[tristate] = 0;

  while(1)
  {
    for(int i = 0; i < 4; i++)
    {
      LEDs[value] = i;
    }
  }

  return 0;
}


make clean exe
Memory utilization:
   text	   data	    bss	    dec	    hex	filename
    428	      0	      0	    428	    1ac	main.elf
Executable (neorv32_exe.bin) size in bytes:
440

mv main.bin blink_upper_LEDs.bin

riscv32-unknown-elf-objdump -D main.elf > blink_upper_LEDs.objdump

bin2coe -i blink_upper_LEDs.bin -o blink_upper_LEDs.coe -w 32



## Overlay handling

The following cells handle instancatiung, programming, and restarting the neo overlay.

In [3]:
from pynq.overlay import Overlay
neo_overlay = Overlay(r"/home/xilinx/neorv32_overlays/rv32i_overlay.bit")

from pynq import GPIO

neo_reset_n = GPIO(GPIO.get_gpio_pin(0), 'out')
neo_reset_n.write(1)


In [4]:
# Wrtie bin file to memory
little_endian = False
BRAM_data = [] 
with open("blink_upper_LEDs.bin", "rb") as f:
    addr = 0
    raw_bytes = raw_bytes = f.read(4)
    while raw_bytes != b"":
        if little_endian:
            BRAM_data.append(
                sum([
                    raw_bytes[0] << 3*8,
                    raw_bytes[1] << 2*8,
                    raw_bytes[2] << 1*8,
                    raw_bytes[3] << 0*8,
                ])
            )   
        else:
            BRAM_data.append(
                sum([
                    raw_bytes[3] << 3*8,
                    raw_bytes[2] << 2*8,
                    raw_bytes[1] << 1*8,
                    raw_bytes[0] << 0*8,
                ])
            )

        addr += 4
        raw_bytes = f.read(4)

#for addr, data in enumerate(BRAM_data):#
#    print(hex(4*addr)[2:], hex(data)[2:])

neo_reset_n.write(0)

for addr, word in enumerate(BRAM_data):
    neo_overlay.pynq_BRAM_controller.write(4*addr, word)

neo_reset_n.write(1)

In [5]:
# Neo kickoff

neo_reset_n.write(0)

for _ in range(2**10):
    pass
    
neo_reset_n.write(1)